# Introduction to Python: Pandas

In [ ]:
import pandas as pd

## Reading CSV File

In [ ]:
CSV_IN_FILE = '../inFiles/randomuser5000.csv'

In [ ]:
# view the documentation for a function or method
pd.read_csv?
# pd.read_csv??
# pd.read_csv(      Shift + Tab

In [ ]:
# read data from a CSV file into a DataFrame
df = pd.read_csv(CSV_IN_FILE)
df.shape

## Reading JSON File

In [ ]:
JSON_IN_FILE = '../inFiles/flights.json'

In [ ]:
# read non-nested data from a JSON file into a DataFrame
df = pd.read_json(JSON_IN_FILE)
df.shape

In [ ]:
JSON_IN_FILE_NESTED = '../inFiles/randomuser5000.json'

In [ ]:
import json
from pandas.io.json import json_normalize

In [ ]:
# read nested data from a JSON file into a DataFrame
with open(JSON_IN_FILE_NESTED, 'r', newline='') as in_file:  
    data = json.load(in_file)
    records = data['results']
    df = json_normalize(records)
df.shape

## Reading Excel File

In [ ]:
EXCEL_IN_FILE = '../inFiles/randomuser5000.xlsx'

In [ ]:
# read data from an Excel worksheet into a DataFrame
df = pd.read_excel(EXCEL_IN_FILE, sheet_name='randomuser5000_all')
df.shape

## Viewing Data

In [ ]:
# view info about columns, including names, counts, and data types
df.info()

In [ ]:
# view first few rows of data
df.head()

In [ ]:
# view last few rows of data
df.tail()

In [ ]:
# view DataFrame's index
df.index

In [ ]:
# view column names
df.columns

## Selecting Data

### By Index Position

In [ ]:
# select rows by index position
df.iloc[0:4,:]

In [ ]:
# select columns by index position
df.iloc[:,[5,6]].head()

In [ ]:
# select rows and columns by index position
df.iloc[[0,1],[5,6]]

### By Label

In [ ]:
# select columns by label
df.loc[:,['location.city', 'location.state']].head()

#### Set Index

In [ ]:
# set location.state as index so we can select rows by label
df.set_index('location.state', inplace=True, drop=True)
df.head()

#### Select Rows by Index Label

In [ ]:
# select rows by index label
df.loc['california',:]

In [ ]:
# select rows and columns by label
df.loc[['maryland', 'virginia'],['name.first', 'name.last', 'location.city']]

#### Reset Index

In [ ]:
# reset the index
df.reset_index(inplace=True)
df.head()

#### Set DatetimeIndex

In [ ]:
# set column 'dob' as index 
df.set_index(pd.DatetimeIndex(df['dob']), inplace=True)
df.drop(['dob'], axis=1, inplace=True)
df.head()

#### Select Rows by DatetimeIndex

In [ ]:
# select rows and columns based on datetime index
df.loc['1980-10',['name.first', 'name.last', 'location.state']]

#### Reset Index

In [ ]:
# reset the index
df.reset_index(inplace=True)

### Select Rows Where Value in Column Meets a Condition

#### Dictionary syntax to specify column

In [ ]:
# dictionary syntax to specify column, in case the column name contains spaces, periods, or other troublesome characters
# value in column equals a specific string
df.loc[df['gender'] == 'female',:]

#### Dot syntax to specify column

In [ ]:
# dot syntax to specify column that doesn't contain spaces
# value in column equals a specific string
df.loc[df.gender == 'female',:]

In [ ]:
# value in column is greater than a specific integer
df.loc[df['location.postcode'] > 83000,:]

In [ ]:
# convert value in column to datetime
# value in column is later (more recent) than a specific date
df.loc[pd.to_datetime(df.dob) > '1995-03',:]

### Select Rows Where Value in Column Is In a Set of Interest

In [ ]:
# our set of interest
western_states = ['washington', 'oregon', 'california']

# value in column IS in our set of interest
df.loc[df['location.state'].isin(western_states),:]

### Select Rows Where Value in Column Is NOT In a Set of Interest

In [ ]:
# tilda specifies 'NOT'

# value in column IS NOT in our set of interest
df.loc[~df['location.state'].isin(western_states),:]

### Select Rows Where Value in Column Matches a Pattern

In [ ]:
# value in column 'startswith'
df.loc[df['name.last'].str.startswith('mc'),:]

In [ ]:
# value in column 'endswith'
df.loc[df['location.city'].str.endswith('burg'),:]

In [ ]:
# value in column 'contains'
df.loc[df['location.postcode'].astype('str').str.contains('488'),:]

### Specify Multiple Conditions

In [ ]:
# & means AND
# | means OR

# a combination of row conditions: 
# rows where gender is female AND (either postcode > 95000 or dob > 1995-03)
row_mask = ((df.gender == 'female') & 
            ((df['location.postcode'] > 95000) | (pd.to_datetime(df.dob) > '1995-03')))

df.loc[row_mask,:]

In [ ]:
# a combination of row conditions:
# rows where city starts with po AND state is in western_states AND dob > 1958-03
row_mask = ((df['location.city'].str.startswith('po')) & 
            (df['location.state'].isin(western_states)) & 
            (pd.to_datetime(df.dob) > '1958-03'))

df.loc[row_mask,:]

### Unique Values in Column

In [ ]:
df.gender.unique()

In [ ]:
sorted(list(df.gender.unique()), reverse=True)

### Value Counts

In [ ]:
df.gender.value_counts()

In [ ]:
df.gender.value_counts(sort=False)

In [ ]:
# convert to fraction of records with each category value, expressed as decimal
df.gender.value_counts(normalize=True)

## Munging Data

### Apply Function to Rows and Columns

In [ ]:
# apply function to single column, a Series, with apply
df['location.postcode'].apply(lambda x: str(x) + '-0000').head()

In [ ]:
# apply function to multiple columns, a DataFrame, with applymap
df[['name.first', 'name.last']].applymap(lambda x: x.upper()).head()

### Create a New Column

#### Address column

In [ ]:
# create a new column 'address'
address_components = ['location.street', 'location.city', 'location.state', 'location.postcode']
df['address'] = df[address_components].apply(lambda x: ' '.join(x.astype(str)), axis=1) # astype(str) because postcode is integer

# view the new column and the columns used to create it
df[['location.street', 'location.city', 'location.state', 'location.postcode', 'address']].head(10)

#### Phone Number column

In [ ]:
# create a new column 'phone_number'
strip_to_int = lambda x: int(x.replace('(', '').replace(')', '').replace('-', '')) # remove punctuation and convert to integer
df['phone_number'] = df.phone.apply(strip_to_int)

# view the new column and the column used to create it
df[['phone', 'phone_number']].head()

#### Age column

In [ ]:
from datetime import date

In [ ]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [ ]:
# convert date of birth (dob) column to datetime
df.dob = pd.to_datetime(df.dob)

# calculate the person's age in years and add the value as a column
df['age'] = df.dob.apply(lambda x: calculate_age(x))

# view the new column and the column used to create it
df[['dob', 'age']].head()

## Grouping Data

### Group by single column

In [ ]:
# group data by state
by_state = df.groupby('location.state')

# view group names and the data for the group named 'wyoming'
for name, group in by_state:
    print(name)
    if name == 'wyoming':
        print(group)

In [ ]:
# display descriptive statistics for postcode by state
df.groupby('location.state')[['location.postcode']].describe().round(2)

In [ ]:
# display specific percentile values for postcode by gender
df.groupby('gender')[['location.postcode']].quantile([0.1, 0.5, 0.9]).unstack().round(2)

In [ ]:
# display mean and standard deviation of postcode by gender
df.groupby('gender')[['location.postcode']].agg(['mean', 'std']).round(2)

In [ ]:
# display mean and standard deviation of postcode
# and count, minimum, and maximum of phone_number by gender
df.groupby('gender').agg({'location.postcode' : ['mean', 'std'], 
                          'phone_number' : ['count', 'min', 'max']}).round(2)

### Group by multiple columns

In [ ]:
by_gender_and_state = df.groupby(['gender', 'location.state'])

# display mean of postcode by gender and state
by_gender_and_state[['location.postcode']].mean().round(2).unstack().T.head() # .head() to shorten the output

### Group by DatetimeIndex

#### Set DatetimeIndex

In [ ]:
# set column 'registered' as index 
df.set_index(pd.DatetimeIndex(df['registered']), inplace=True)
df.drop(['registered'], axis=1, inplace=True)

#### Resample / Group by DatetimeIndex

In [ ]:
# count the number of people who registered by year
df.resample('AS')['name.first', 'name.last'].count()

#### Reset Index

In [ ]:
# reset the index
df.reset_index(inplace=True)

### Group by State and DatetimeIndex

In [ ]:
# count the number of people who registered by state and year
df.set_index('registered').groupby('location.state')['name.first', 'name.last'].resample("AS").count()

In [ ]:
df.groupby(['location.state', pd.Grouper(key='registered', freq='AS')])['location.postcode'].sum()

#### Reset Index

In [ ]:
# reset the index
df.reset_index(inplace=True)

## Reshaping Data

### Pivot Table

In [ ]:
# index specifies the column(s) with the values that will become the rows
# columns specifies the column(s) with the values that will become the columns
# values specifies the column with the values that will become the data

# aggfunc='mean' is the default, so you don't need to include it if you're calculating means
df.pivot_table(index=['location.state'], 
               columns=['gender'], 
               values=['phone_number'], 
               aggfunc='mean').head() # .head() to shorten the output

In [ ]:
import numpy as np

In [ ]:
# states become the rows
# genders and titles become the columns
# postcodes become the data

# np.sum refers to 'import numpy as np'
# aggregation function is np.sum, so sum the postcode values for each category
df.pivot_table(index=['location.state'], 
               columns=['gender', 'name.title'], 
               values=['location.postcode'], 
               aggfunc=np.sum).head() # .head() to shorten the output

### Stack and Unstack

#### Unstack Example

In [ ]:
# original result of operations
df.groupby('gender')[['location.postcode']].quantile([0.1, 0.5, 0.9]).round(2)

In [ ]:
# unstack
df.groupby('gender')[['location.postcode']].quantile([0.1, 0.5, 0.9]).unstack().round(2)

#### Stack Example

In [ ]:
# original result of operations, note the transpose 'T'
df.groupby('gender')[['location.postcode']].quantile([0.1, 0.5, 0.9]).T.round(2)

In [ ]:
# stack
df.groupby('gender')[['location.postcode']].quantile([0.1, 0.5, 0.9]).T.stack().round(2)

### Melt

In [ ]:
# a small DataFrame for examples
small_df = pd.DataFrame({'first' : ['John', 'Mary'], 
                         'last' : ['Doe', 'Bo'], 
                         'height' : [5.5, 6.0], 
                         'weight' : [130, 150]})
small_df

In [ ]:
# specify the columns on which to melt
small_df.melt(id_vars=['first', 'last'])

In [ ]:
# specify the variable and value column names
small_df.melt(id_vars=['first', 'last'], 
              var_name='var', 
              value_name='val')

## Ordering / Sorting Data

In [ ]:
df.head()

In [ ]:
df.sort_values(by=['location.state', 'location.postcode']).head()

In [ ]:
df.sort_values(by=['location.state', 'location.postcode'], ascending=[False, False]).head()

## Merging Data

### Concat

In [ ]:
# concatenate DataFrames vertically, retain indexes
small_df2 = pd.concat([small_df, small_df])
small_df2

In [ ]:
# concatenate DataFrames vertically, create new index
small_df3 = pd.concat([small_df, small_df], ignore_index=True)
small_df3

In [ ]:
# concatenate DataFrames horizontally
small_df4 = pd.concat([small_df, small_df], axis=1)
small_df4

### Merge / Join

In [ ]:
# a tiny DataFrame for examples
tiny_df1 = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
tiny_df1

In [ ]:
# a tiny DataFrame for examples
tiny_df2 = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})
tiny_df2

In [ ]:
# database-style merge of two DataFrames into a new DataFrame
# how='inner' is default
tiny_df = pd.merge(tiny_df1, tiny_df2, how='inner', on='key')
tiny_df

## Plotting Data

In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.style.available

In [ ]:
mpl.style.use('tableau-colorblind10')

### Timeseries

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2018', periods=1000))
ts = ts.cumsum()
df1 = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list('ABCD'))
df1 = df1.cumsum()

df1.plot(figsize=(10,6), legend=False);

### Bar plot

In [ ]:
df1.iloc[5].plot.bar(figsize=(10,6)); plt.axhline(0, color='k');

In [ ]:
df2 = pd.DataFrame(np.random.rand(10, 4), columns=['a', 'b', 'c', 'd'])

df2.plot.bar(stacked=True, figsize=(10,6));

### Histogram

In [ ]:
df3 = pd.DataFrame({'a': np.random.randn(1000) + 1, 
                    'b': np.random.randn(1000), 
                    'c': np.random.randn(1000) - 1}, 
                   columns=['a', 'b', 'c'])

df3.plot.hist(alpha=0.5, bins=20, figsize=(10,6));

### Scatter plot

In [ ]:
df4 = pd.DataFrame(np.random.rand(50, 4), columns=['a', 'b', 'c', 'd'])

ax = df4.plot.scatter(x='a', y='b', color='Blue', label='Group 1', figsize=(10,6));
df4.plot.scatter(x='c', y='d', color='Orange', label='Group 2', ax=ax);

### Plot with Seaborn

In [ ]:
import seaborn as sns

In [ ]:
iris = sns.load_dataset("iris")

sns.pairplot(iris, hue='species', height=2.5);

## Writing CSV File

In [ ]:
# re-read data into a DataFrame to ensure we have clean data to write into CSV, JSON, and Excel files
df = pd.read_csv(CSV_IN_FILE)

# select rows where gender equals male so these output files are different than the ones from Python Basics
males = df.loc[df.gender == 'male',:]
males

In [ ]:
CSV_OUT_FILE = '../outFiles/randomuser5000_males.csv'

In [ ]:
males.to_csv(CSV_OUT_FILE, index=False)
print('Finished writing file.')

## Writing JSON File

In [ ]:
JSON_OUT_FILE = '../outFiles/randomuser5000_males.json'

In [ ]:
males.to_json(JSON_OUT_FILE, orient='records')
print('Finished writing file.')

## Writing Excel File

In [ ]:
EXCEL_OUT_FILE = '../outFiles/randomuser5000_males.xls'

In [ ]:
males.to_excel(EXCEL_OUT_FILE, sheet_name='randomuser_males', index=False)
print('Finished writing file.')